# Import
- 외우지 않아도 됨.

In [1]:
# 셀레니움 : 자동화된 웹 브라우저 테스트
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

# 3.11까지 호환 개발, But 3.12에서도 문제가 없음
from webdriver_manager.chrome import ChromeDriverManager


# 시간 라이브러리 
import arrow
import time
import datetime


# 파이썬 기본 라이브러리
import os
import re

# Request 요청과 관련된 라이브러리
import requests

# HTML DOM 데이터를 -> TEXT 데이터 형식으로 바꾸어 볼 수 있는 라이브러리 
from bs4 import BeautifulSoup






# Driver 생성
- chrome driver(크롬 브라우저의 핵심을 모방한 프로그램, chrome 이 아니더라도 edge, firefox 등 다른 브라우저도 모방 가능)

In [3]:

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument("window-size=1920,1024")
# 서버에서 user_agent로 봇인지 사람인지 판단하는 경우가 있기 때문에, 자주 사용되는 user_agent들 중 하나를 사용
# https://techblog.willshouse.com/2012/01/03/most-common-user-agents/
chrome_options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:138.0) Gecko/20100101 Firefox/138.0") 
chrome_prefs = {
    "credentials_enable_service": False,
    "profile.password_manager_enabled": False,
    "profile.password_manager_leak_detection": False,  # 로그인 유출 경고창 비활성화
}
chrome_options.add_experimental_option("prefs", chrome_prefs)


service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 로그인

In [2]:

# 민감한 정보는 환경변수에 등록, 등록한 환경변수를 불러오는 코드
login_id = os.environ.get("LOGIN_ID")
login_pw = os.environ.get("LOGIN_PW")
login_url = "https://www.incheoncc.com:1436/login/login.asp?returnurl=/pagesite/reservation/live.asp?"
driver.get(login_url)

# 개발자도구 - element - id 찾기, 
# shift+tab으로 함수/메소드 정의 시그니처 확인
id_element = driver.find_element(value="login_id")
pw_element = driver.find_element(value="login_pw")
id_element.send_keys(login_id)
pw_element.send_keys(login_pw)
login_button = driver.find_element(value="bt_login",by=By.CLASS_NAME)
login_button.click()
if EC.alert_is_present():
    result = driver.switch_to.alert
    result.accept()

NameError: name 'driver' is not defined

# 모니터링
- cursor AI를 프롬프트 이용하여 프로그래밍
- 서버에서 코스를 9시에 오픈하기 때문에 정확히 9시에 예약 가능을 확인하는 모니터링이 실행되어야 해. 
- but 안전하게 9시보다 조금 일찍 실행되어야 하고, 또 너무 일찍 실행되어서 서버에게 지속된 요청으로 매크로가 아니여야함.
- 적당히 8시 59분 50초로 기준 설정, 즉 8시 59분 50초 이전에는 1분 단위로 서버에게 요청해야해
- 유저는 8시 50분 - 9시 사이에 실행한다고 가정하고, 10분 이상 예약가능 모니터링을 서버에게 요청하지마.
- monitor.py 파일에 클래스와 메소드를 만들어 줘.

구체적인 예시
모니터링 첫 시도: 8시 56분 10초 \
모니터링 두번째 시도: 8시 57분 10초\
모니터링 3번째 시도: 8시 58분 10초\
모니터링 4번째 시도: 8시 59분 10초\
모니터링 5번째 시도: 8시 59분 50초\
모니터링 6번째 시도: 8시 59분 51초\
모니터링 7번째 시도: 8시 59분 52초\
...




In [87]:
from typing import Optional
class GolfReservationMonitor:
    def __init__(self, selenium_cookies):
        self.hour = 8
        self.minute = 59
        self.second = 50
        self.base_url = "https://www.incheoncc.com:1436"
        self.session = requests.Session()
        # 기본 헤더 설정 (실제 브라우저처럼 보이게)
        self.session.headers.update(
            {
                "User-Agent": get_random_user_agent(),
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
                "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3",
                "Accept-Encoding": "gzip, deflate, br",
                "Connection": "keep-alive",
                "Upgrade-Insecure-Requests": "1",
            }
        )

        for cookie in selenium_cookies:
            self.session.cookies.set(cookie["name"], cookie["value"])

    def check_time_window(self) -> tuple[bool, int]:
        """
        현재 시간에 따른 모니터링 가능 여부와 대기 시간 반환

        Returns:
            tuple[bool, int]: (모니터링 가능 여부, 다음 체크까지 대기 시간(초))

        시간대별 모니터링 주기:
        - ~ 8시 59분 50초 : 1분에 1번 (60초 간격)
        - 8시 59분 50초 ~ : 1초에 1번 (1초 간격)
        """
        # 지금 시간
        now = datetime.datetime.now()

        # 바뀔 시간 기준의 정의
        criterion = now.replace(
            hour=self.hour, minute=self.minute, second=self.second, microsecond=0
        )

        if now < criterion:
            # ~ 기준 시간 : 1분 간격
            return True, 60
        elif criterion <= now:
            # 기준 시간 ~ : 1초 간격
            return True, 1
        else:
            return False, 0

    def get_calendar_data(self, yyyymmdd: str) -> Optional[bool]:
        """
        특정 날짜의 예약 상태를 확인
        yyyymmdd: YYYYMMDD 형식
        Returns: True if live, False if not live, None if error
        """
        try:
            # 달력 데이터를 가져오기 위한 AJAX 요청
            calendar_url = (
                f"{self.base_url}/GolfRes/onepage/real_calendar_ajax_view.asp"
            )

            # 타겟 날짜에서 년월 추출
            yyyymm = yyyymmdd[:6]  # YYYYMM
            now_yyyymm = datetime.datetime.now().strftime("%Y%m")
            calnum = "1" if yyyymm == now_yyyymm else "2"
            # POST 데이터 준비
            post_data = {
                "golfrestype": "real",
                "schDate": yyyymm,
                "usrmemcd": "12",  # 유저 멤버 번호
                "toDay": yyyymmdd,
                "calnum": calnum,
            }

            response = self.session.post(calendar_url, data=post_data)
            response.raise_for_status()

            # HTML 파싱
            soup = BeautifulSoup(response.text, "html.parser")

            # 해당 날짜의 링크 찾기
            target_day = int(yyyymmdd[-2:])  # 일자만 추출 01 -> 1

            # 모든 날짜 링크 찾기
            date_links = soup.find_all("a", href=True)
            for link in date_links:
                # 링크 텍스트가 해당 일자와 일치하는지 확인
                if link.get_text().strip() == str(target_day):
                    if "예약가능" == link.get("title", ""):  # type: ignore
                        return True
            return False

        except Exception as e:
            logger.info(f"오류 발생: {e}")
            return None

    def monitor_is_alive_date(self, yyyymmdd: str, timeout_minutes: int = 10) -> bool:
        """
        원하는 날짜가 live 상태가 될 때까지 시간대별 모니터링

        Args:
            yyyymmdd (str): 모니터링할 날짜 (YYYYMMDD 형식)
            timeout_minutes (int): 최대 모니터링 시간 (분 단위, 기본값: 10분)

        Returns:
            bool: True if 날짜가 live 상태가 됨, False if 시간 초과 또는 모니터링 불가

        모니터링 주기:
        - ~ 8시 59분 50초: 1분 간격
        - 8시 59분 50초 ~ : 1초 간격
        """

        start_time = datetime.datetime.now()
        timeout_seconds = timeout_minutes * 60

        logger.info(f"🏌️ 골프 예약 모니터링 시작")
        logger.info(f"📅 대상 날짜: {yyyymmdd}")
        logger.info(f"⏰ 모니터링 시간:")
        logger.info(f"   • 00:00 ~ 08:59:50 → 1분 간격")
        logger.info(f"   • 08:59:50 ~ 24:00 → 1초 간격")
        logger.info(f"   • 최대 모니터링 시간: {timeout_minutes}분")
        logger.info(f"   • Ctrl+C로 언제든 중단 가능")
        logger.info("-" * 50)

        check_count = 0
        last_check_time = None

        try:
            while True:
                # 타임아웃 체크
                current_time = datetime.datetime.now()
                elapsed_seconds = (current_time - start_time).total_seconds()
                if elapsed_seconds >= timeout_seconds:
                    logger.info(
                        f"⏰ {timeout_minutes}분 타임아웃으로 모니터링을 종료합니다"
                    )
                    return False

                # 현재 시간 확인 및 모니터링 가능 여부 체크
                can_monitor, wait_seconds = self.check_time_window()

                # 중복 체크 방지 (같은 시간대에 여러 번 체크하지 않음) 8시에 확인 필요
                if wait_seconds == 60:
                    # 다음 체크 예정 시간이 8:59:50을 넘기면, 8:59:50에 맞춰서 sleep
                    now = datetime.datetime.now()
                    next_check = now + datetime.timedelta(seconds=wait_seconds)
                    switch_time = now.replace(
                        hour=self.hour,
                        minute=self.minute,
                        second=self.second,
                        microsecond=0,
                    )
                    # 다음 체크 > 기준 시간 > 지금
                    if next_check > switch_time > now:
                        # 8:59:50까지 남은 초만큼 sleep
                        sleep_seconds = (switch_time - now).total_seconds()
                        time.sleep(sleep_seconds)
                    # 1분 & 기준 시간 > 다음 체크 > 지금
                    else:
                        time.sleep(wait_seconds)
                # 1초 일때
                else:
                    time.sleep(wait_seconds)

                # 모니터링 실행
                check_count += 1
                time_str = current_time.strftime("%H:%M:%S")
                interval_str = "1분 간격" if wait_seconds == 60 else "1초 간격"

                logger.info(f"🔍 검사 #{check_count} - {time_str} ({interval_str})")

                # 예약 상태 확인
                is_live = self.check_calendar_data(yyyymmdd)

                if is_live is None:
                    logger.info("❌ 데이터 조회 실패")
                elif is_live:
                    logger.info(f"✅ 성공! {yyyymmdd} 날짜가 예약 가능 상태입니다!")
                    return True
                else:
                    logger.info(f"⏳ {yyyymmdd} 아직 예약 불가능...")

        except KeyboardInterrupt:
            logger.info("🛑 사용자가 모니터링을 중단했습니다 (Ctrl+C)")
            return False

    def check_calendar_data(self, yyyymmdd: str) -> Optional[bool]:
        """실제 달력 데이터 확인 (간소화된 버전)"""
        try:
            return self.get_calendar_data(yyyymmdd)

        except Exception as e:
            logger.info(f"확인 중 오류: {e}")
            return None


In [3]:
# 모니터링 테스트/디버깅 코드 
def main():
    """메인 실행 함수"""
    
    selenium_cookies = driver.get_cookies()
    monitor = GolfReservationMonitor(selenium_cookies)
    
    # 사용자로부터 날짜 입력받기
    print("=" * 60)
    print("🏌️  인천국제CC 골프 예약 모니터링 시스템")
    print("=" * 60)
    
    while True:
        try:
            date_input = input("\n📅 모니터링할 날짜를 입력하세요 (YYYY-MM-DD 형식): ").strip()
            
            # 날짜 형식 검증
            target_datetime = datetime.datetime.strptime(date_input, "%Y-%m-%d")
            target_date = target_datetime.strftime("%Y%m%d")
            
            # 과거 날짜 체크
            if target_datetime.date() < datetime.date.today():
                print("❌ 과거 날짜는 선택할 수 없습니다.")
                continue
                
            break
            
        except ValueError:
            print("❌ 잘못된 날짜 형식입니다. YYYY-MM-DD 형식으로 입력해주세요.")
            continue
    
    print(f"\n🎯 선택된 날짜: {date_input} ({target_date})")       

    
    # 모니터링 시작
    print("\n🚀 모니터링을 시작합니다...")
    
    try:
        result = monitor.monitor_is_alive_date(target_date)
        
        if result:
            print(f"\n🎉 성공! {date_input} 날짜 예약이 가능해졌습니다!")
            print("💡 지금 바로 예약 사이트로 이동하세요!")
        else:
            print(f"\n😞 아쉽게도 {date_input} 날짜가 예약 가능 상태로 변경되지 않았습니다.")
            print("💡 다음 기회에 다시 시도해보세요.")
            
    except KeyboardInterrupt:
        print("\n\n⏹️  사용자에 의해 모니터링이 중단되었습니다.")
    except Exception as e:
        print(f"\n❌ 예상치 못한 오류가 발생했습니다: {e}")


In [89]:
main()

🏌️  인천국제CC 골프 예약 모니터링 시스템



📅 모니터링할 날짜를 입력하세요 (YYYY-MM-DD 형식):  2025-07-11



🎯 선택된 날짜: 2025-07-11 (20250711)

🚀 모니터링을 시작합니다...
🏌️ 골프 예약 모니터링 시작
📅 대상 날짜: 20250711
⏰ 모니터링 시간:
   • 00:00 ~ 08:59:50 → 1분 간격
   • 08:59:50 ~ 24:00 → 1초 간격
   • 최대 모니터링 시간: 10분
   • Ctrl+C로 언제든 중단 가능
--------------------------------------------------
🔍 검사 #1 - 22:10:59 (1초 간격)
✅ 성공! 20250711 날짜가 예약 가능 상태입니다!

🎉 성공! 2025-07-11 날짜 예약이 가능해졌습니다!
💡 지금 바로 예약 사이트로 이동하세요!


In [90]:
# StaleElementReferenceException
# InvalidSessionIdException

# 리스트 페이지에서 해당 캘린더 날짜 클릭하는 코드 

In [4]:
# 예약 페이지에서 해당 캘린더 날짜 클릭하는 코드 
yyyy_mm_dd = "20250711"
wait = WebDriverWait(driver, 10)
cal_live_dates = wait.until(
    EC.presence_of_all_elements_located(
        (
            By.XPATH,
            "//table[@class='cm_calender_tbl']//td/a[contains(@class,'cal_live')]",
        )
    )
)
# 날짜 클릭
for cal_live_date in cal_live_dates:
    href = cal_live_date.get_attribute("href")
    if yyyy_mm_dd in href:
        cal_live_date.click()

NameError: name 'driver' is not defined

In [92]:
# 코스 개수 확인
len(driver.find_elements(value="//table[@class='cm_time_list_tbl']/tbody/tr",by=By.XPATH))

14

In [5]:
# 사용할 비즈니스 데이터(엔티티) 소개드린 DDD 라는 방식에서 언급
from enum import Enum
from dataclasses import dataclass

class OutInType(Enum):
    OUT = 1  # 아웃 코스
    IN = 2  # 인 코스
@dataclass
class Course:
    point_id_out_in: OutInType
    course_type: str
    time: str

@dataclass
class TimePoint:
    hour: int
    minute: int

    def __post_init__(self):
        if not (0 <= self.hour <= 23):
            raise ValueError("hour는 0~23 사이여야 합니다.")
        if not (0 <= self.minute <= 59):
            raise ValueError("minute는 0~59 사이여야 합니다.")

    def strf_hhmm(self):
        return f"{self.hour:02d}{self.minute:02d}"

@dataclass
class TimeRange:
    start: TimePoint
    end: TimePoint
    priority_time: TimePoint


# 웹 페이지의 코스를 수집해서, 원하는 시간대와 선호하는 시간에 의해 우선순위로 정렬된 코스들 리스트 반환

In [119]:
# 웹 페이지의 코스를 수집해서, 원하는 시간대와 선호하는 시간에 의해 우선순위로 정렬된 코스들 리스트 반환
time_range_model=TimeRange(start=TimePoint(hour=5,minute=0),
                           end=TimePoint(hour=7,minute=0),
                           priority_time=TimePoint(hour=6,minute=0)
                          )

                                           
wait = WebDriverWait(driver, 10)
table = wait.until(
    EC.presence_of_element_located(
        (
            By.CLASS_NAME,
            "cm_time_list_tbl",
        )
    )
)
# 헤더 제외한 행
rows = table.find_elements(By.TAG_NAME, "tr")[1:] 

# 결과를 저장할 list
scrpaed_courses = []

# 각 행을 순회하며 데이터 추출
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    if len(cells) != 7:  # 모든 컬럼이 있는지 확인
        raise Exception("코스의 칼럼이 바뀌었습니다.")
    course_type = cells[1].text
    course_time = cells[2].text
    # 문자열을 OutInType enum으로 변환
    point_id_out_in = (
        OutInType.OUT if course_type == "OUT" else OutInType.IN
    )
    scrpaed_courses.append(Course(point_id_out_in=point_id_out_in,course_type=course_type, time=course_time))

def time_to_minutes(tstr):
    h, m = map(int, tstr.split(":"))
    return h * 60 + m

start_minutes = time_range_model.start.hour * 60 + time_range_model.start.minute
end_minutes = time_range_model.end.hour * 60 + time_range_model.end.minute
priority_minutes = (
    time_range_model.priority_time.hour * 60
    + time_range_model.priority_time.minute
)
filtered = [
    course
    for course in scrpaed_courses
    if time_to_minutes(course.time) >= start_minutes
    and time_to_minutes(course.time) <= end_minutes
]
sorted_course_times = sorted(
    filtered,
    key=lambda x: abs(time_to_minutes(x.time) - priority_minutes),
)
sorted_course_times

[Course(point_id_out_in=<OutInType.OUT: 1>, course_type='OUT', time='06:00'),
 Course(point_id_out_in=<OutInType.OUT: 1>, course_type='OUT', time='05:42'),
 Course(point_id_out_in=<OutInType.IN: 2>, course_type='IN', time='05:42'),
 Course(point_id_out_in=<OutInType.OUT: 1>, course_type='OUT', time='05:36'),
 Course(point_id_out_in=<OutInType.IN: 2>, course_type='IN', time='05:36'),
 Course(point_id_out_in=<OutInType.OUT: 1>, course_type='OUT', time='05:30'),
 Course(point_id_out_in=<OutInType.IN: 2>, course_type='IN', time='05:30'),
 Course(point_id_out_in=<OutInType.OUT: 1>, course_type='OUT', time='05:24'),
 Course(point_id_out_in=<OutInType.IN: 2>, course_type='IN', time='05:24'),
 Course(point_id_out_in=<OutInType.OUT: 1>, course_type='OUT', time='05:18'),
 Course(point_id_out_in=<OutInType.IN: 2>, course_type='IN', time='05:18'),
 Course(point_id_out_in=<OutInType.OUT: 1>, course_type='OUT', time='05:12')]

In [123]:
# 리스트보다 queue라는 자료구조(선입선출)를 사용하는게 예약하는 task에 직관적임, deque는 양방향 queue임, 정확하게는 이해할 필요는 없음

sorted_course_times
if not sorted_course_times:
    logger.info(f"🛑 {yyyy_mm_dd}에 선택한 시간 중 가능한 시간대가 없습니다!")
    raise RuntimeError(f"🛑 {yyyy_mm_dd} 날짜가 예약 불가능 상태입니다!")

from collections import deque

courses_dq = deque(sorted_course_times)

In [121]:
# 없어진다면 index error
course = courses_dq.popleft()

In [122]:
course


Course(point_id_out_in=<OutInType.OUT: 1>, course_type='OUT', time='06:00')

# 리스트 페이지에서 특정 코스의 row의 예약 버튼을 클릭하는 코드 

In [79]:
# 리스트 페이지에서 특정 코스의 row의 예약 버튼을 클릭하는 코드 
# 추후 에러(경쟁 선점에 의한) 처리 필요 
# select_time, course_type(OUT,IN)으로 row 찾기
for course in sorted_course_times:
    wait = WebDriverWait(driver, 10)
    table = wait.until(
        EC.presence_of_element_located(
            (
                By.CLASS_NAME,
                "cm_time_list_tbl",
            )
        )
    )
    # 헤더 제외한 행
    rows = table.find_elements(By.TAG_NAME, "tr")[1:] 
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        if course.course_type == cells[1].text and course.time == cells[2].text:
            cells[6].click()
            break
    break


# 상세 페이지에서 예약 버튼을 누르고 alert을 "네" 처리하는 코드
- 실제 예약되기 때문에 주석 처리함

In [80]:
# 상세 페이지에서 예약 버튼을 누르고 alert을 "네" 처리하는 코드
btn = driver.find_element(By.XPATH, "//form/div/button[1]")
if btn.text == "예약":
    btn.click()
## 주석 해제시 예약 완료 처리 됨
# if EC.alert_is_present():
#     result = driver.switch_to.alert
#     result.accept()

def convert_date_format(date_str):
    try:
        # 날짜 형식이 8자리인지 확인
        if len(date_str) != 8:
            return "올바른 날짜 형식이 아닙니다."
            
        # 연도, 월, 일 추출
        year = date_str[:4]
        month = date_str[4:6]
        day = date_str[6:]
        
        # 변환된 형식으로 반환
        converted_date = f"{year}년 {month}월 {day}일"
        return converted_date
        
    except Exception as e:
        return f"에러가 발생했습니다: {str(e)}"

- date = "20250629"
- result = convert_date_format(date)
- print(result)  # 출력: 2025년 06월 29일
- 이 함수는 다음과 같은 특징이 있습니다:

- 'YYYYMMDD' 형식의 8자리 문자열을 입력받습니다.
- 문자열 슬라이싱을 사용하여 연, 월, 일을 분리합니다.
- 연도: 처음 4자리
- 월: 중간 2자리
- 일: 마지막 2자리
- f-string을 사용하여 "yyyy년 mm월 dd일" 형식으로 변환합니다.
- 예외 처리를 통해 잘못된 입력에 대응합니다.
- 입력된 날짜가 8자리가 아닌 경우 오류 메시지를 반환합니다.

In [129]:
def convert_date_format(date_str):
    try:
        # 날짜 형식이 8자리인지 확인
        if len(date_str) != 8:
            return "올바른 날짜 형식이 아닙니다."
            
        # 연도, 월, 일 추출
        year = date_str[:4]
        month = date_str[4:6]
        day = date_str[6:]
        
        # 변환된 형식으로 반환
        converted_date = f"{year}년 {month}월 {day}일"
        return converted_date
        
    except Exception as e:
        return f"에러가 발생했습니다: {str(e)}"

In [131]:
# 테스트
# yyyymmdd -> yyyy년 mm월 dd일
point_date=convert_date_format("20250629")


Course(point_id_out_in=<OutInType.OUT: 1>, course_type='OUT', time='06:00')

# 예약완료 후, 예약확인 페이지에서 예약완료 여부를 확인하는 코드(날짜,시간, OUT|IN 으로 판별)

In [133]:
# 예약확인 페이지에서 예약완료 여부를 확인하는 코드(날짜,시간, OUT|IN 으로 판별)
course = Course(time="07:06",course_type="OUT", point_id_out_in=1)

table = driver.find_element(By.CLASS_NAME,"cm_time_list_tbl")

# 코드 수정해서 확인
for reservation in table.find_elements(By.TAG_NAME,"tr")[1:]:
    if reservation.find_elements(By.TAG_NAME,"td")[1].text == point_date and \
    reservation.find_elements(By.TAG_NAME,"td")[2].text == course.time and \
    course.course_type  in reservation.find_elements(By.TAG_NAME,"td")[3].text:
        print("예약완료")
        # return True

예약완료
